In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
try:
    import empyrical as emp
except:
    emp = None
import tushare as ts
import time
import random

from common.log import *
from common.config import Config
from spider.spider_nasdaq import Spider_nasdaq
from spider.spider_coinmarketcap import Spider_coinmarketcap

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [ ]:
df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/EOS-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2019-06-04T00%3A00%3A00.000Z')
len(df)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/EOS-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-11-15T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/EOS-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-04-29T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

# _df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/EOS-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2017-11-19T00%3A00%3A00.000Z')
# df = df.append(_df, ignore_index=True)
# time.sleep(1)

df.columns = ['date', 'open', 'high', 'low', 'close', 'volumn']
df['date'] = df['date'].apply(pd.to_datetime)
df['date'] = df['date'] + pd.Timedelta(hours=8)
df.head(5)
df.tail(5)

datafile = '../database/market/EOS_OKEX.csv'
df.to_csv(datafile, header=['date', 'open', 'high', 'low', 'close', 'volume'], index=False, encoding='utf-8')

In [ ]:
# t = pd.to_datetime(df.date[0], utc=True)
# # t
# # t.tz

# df['date'] = df['date'].apply(pd.to_datetime)
# # df.date[0]
# # df.date[0] + pd.Timedelta(hours=8)
# df['date'] = df['date'] + pd.Timedelta(hours=8)
# df['date']
# df

In [ ]:
df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/BTC-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2019-06-04T00%3A00%3A00.000Z')
len(df)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/BTC-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-11-15T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/BTC-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-04-29T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/EOS-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2017-10-11T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

df.columns = ['date', 'open', 'high', 'low', 'close', 'volumn']
df['date'] = df['date'].apply(pd.to_datetime)
df['date'] = df['date'] + pd.Timedelta(hours=8)
df.head(5)
df.tail(5)

datafile = '../database/market/BITCOIN_OKEX.csv'
df.to_csv(datafile, header=['date', 'open', 'high', 'low', 'close', 'volume'], index=False, encoding='utf-8')

In [ ]:
df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/ETH-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2019-06-04T00%3A00%3A00.000Z')
len(df)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/ETH-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-11-15T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/ETH-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-04-29T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/ETH-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2017-10-11T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

df.columns = ['date', 'open', 'high', 'low', 'close', 'volumn']
df['date'] = df['date'].apply(pd.to_datetime)
df['date'] = df['date'] + pd.Timedelta(hours=8)
df.head(5)
df.tail(5)

datafile = '../database/market/ETHEREUM_OKEX.csv'
df.to_csv(datafile, header=['date', 'open', 'high', 'low', 'close', 'volume'], index=False, encoding='utf-8')

In [ ]:
df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/XRP-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2019-06-04T00%3A00%3A00.000Z')
len(df)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/XRP-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-11-15T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/XRP-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-04-29T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/XRP-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2017-10-11T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

df.columns = ['date', 'open', 'high', 'low', 'close', 'volumn']
df['date'] = df['date'].apply(pd.to_datetime)
df['date'] = df['date'] + pd.Timedelta(hours=8)
df.head(5)
df.tail(5)

datafile = '../database/market/RIPPLE_OKEX.csv'
df.to_csv(datafile, header=['date', 'open', 'high', 'low', 'close', 'volume'], index=False, encoding='utf-8')

In [ ]:
df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/LTC-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2019-06-04T00%3A00%3A00.000Z')
len(df)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/LTC-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-11-15T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/LTC-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-04-29T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/LTC-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2017-10-11T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

df.columns = ['date', 'open', 'high', 'low', 'close', 'volumn']
df['date'] = df['date'].apply(pd.to_datetime)
df['date'] = df['date'] + pd.Timedelta(hours=8)
df.head(5)
df.tail(5)

datafile = '../database/market/LITECOIN_OKEX.csv'
df.to_csv(datafile, header=['date', 'open', 'high', 'low', 'close', 'volume'], index=False, encoding='utf-8')